## Thought process - Initial ideas:

### 1. Preprocessing personal_info.tsv:

Convert birth date to age (today's date - birth date). Check whether customer is Senior (age > 60).  
Encode gender to numerical, since it's categorical. Onehot encoding not needed, because there are only 2 values.  
Convert subscription period to duration in days.  
<b>Output vector: id, age, gender, subsctiption_duration.</b>

### 2. Preprocessing phone_calls.tsv, sms.tsv and data.tsv:

Get total minutes/sms/data, in and out of roaming, towards out/other operator.  
<b>Output vector: id, total_roaming, total_out_of_roaming_our_operator, total_out_of_roaming_other_operator.</b>  
For data.tsv we only have data in/out of roaming.

### 3. Preprocessing invoice.tsv

Get average invoice paid per user, per month.  
Get last non-zero invoice.  
In case Invoice=0, the subscription has been terminated, so Churn=1.  
<b>Output vector: id, average_invoice, last_invoice, churn.</b>

### 4. Preprocessing contract_info.tsv

Get number of contracts per user.
<b>Output vector: id, number_of_contracts</b>


### Final table/vector:
* id
* age
* gender
* subscription_duration
* number_of_contracts
* total_minutes_in_roaming
* total_minutes_out_of_roaming_our_operator
* total_minutes_out_of_roaming_other_operator
* total_sms_in_roaming
* total_sms_out_of_roaming_our_operator
* total_sms_out_of_roaming_other_operator
* total_data_in_roaming
* total_data_out_of_roaming
* average_invoice
* last_invoice
* churn

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
import glob

In [2]:
ls data

 Volume in drive C has no label.
 Volume Serial Number is B6AA-C15A

 Directory of C:\DEV\Projects\Churn task\data

05.08.2021  18:59    <DIR>          .
05.08.2021  18:59    <DIR>          ..
17.11.2019  17:41                 0 .gitkeep
17.11.2019  17:41            29.971 contract_info.tsv
17.11.2019  17:41        16.877.631 data.tsv
17.11.2019  17:41           532.965 invoice.tsv
17.11.2019  17:41            38.890 personal_info.tsv
17.11.2019  17:41        24.662.090 phone_calls.tsv
17.11.2019  17:41        19.785.766 sms.tsv
               7 File(s)     61.927.313 bytes
               2 Dir(s)  74.967.261.184 bytes free


In [3]:
pwd = os.getcwd()
pwd = os.path.join(pwd, "data")
pwd

'C:\\DEV\\Projects\\Churn task\\data'

In [4]:
filepath_personal_info = os.path.join(pwd, "personal_info.tsv")
filepath_contract_info = os.path.join(pwd, "contract_info.tsv")
filepath_phone_calls   = os.path.join(pwd, "phone_calls.tsv")
filepath_sms           = os.path.join(pwd, "sms.tsv")
filepath_data          = os.path.join(pwd, "data.tsv")
filepath_invoice       = os.path.join(pwd, "invoice.tsv")

## Preprocess Personal info

In [5]:
personal_info_column_names = ["id", "birth_date", "gender", "subscription_start_date", "subscription_end_date"]
personal_info_df = pd.read_csv(filepath_personal_info,  sep='\t', names=personal_info_column_names)
personal_info_df

,id,birth_date,gender,subscription_start_date,subscription_end_date
0,0,30.10.1986,F,18.08.2010,01.01.2018
1,1,22.03.1967,M,07.05.1997,01.01.2018
2,2,25.12.1965,M,23.05.2003,01.01.2018
3,3,17.06.1988,F,14.09.2013,01.01.2018
4,4,18.12.1964,M,28.01.2004,01.01.2018
...,...,...,...,...,...
995,995,08.11.1962,M,31.05.2003,01.01.2018
996,996,12.10.1987,F,18.11.2007,01.01.2018
997,997,24.10.1989,F,24.10.2007,13.12.2017
998,998,04.04.1980,F,10.03.2013,01.01.2018


In [6]:
personal_info_df.isna().sum()

id                         0
birth_date                 0
gender                     0
subscription_start_date    0
subscription_end_date      0
dtype: int64

In [7]:
def calculate_age(birth):
    birth = datetime.strptime(birth, "%d.%m.%Y")
    today = datetime.today()
    return today.year - birth.year - ((today.month, today.day) < (birth.month, birth.day))
calculate_age('13.04.1997')

24

In [8]:
personal_info_df['age'] = personal_info_df['birth_date'].apply(calculate_age)
personal_info_df

,id,birth_date,gender,subscription_start_date,subscription_end_date,age
0,0,30.10.1986,F,18.08.2010,01.01.2018,34
1,1,22.03.1967,M,07.05.1997,01.01.2018,54
2,2,25.12.1965,M,23.05.2003,01.01.2018,55
3,3,17.06.1988,F,14.09.2013,01.01.2018,33
4,4,18.12.1964,M,28.01.2004,01.01.2018,56
...,...,...,...,...,...,...
995,995,08.11.1962,M,31.05.2003,01.01.2018,58
996,996,12.10.1987,F,18.11.2007,01.01.2018,33
997,997,24.10.1989,F,24.10.2007,13.12.2017,31
998,998,04.04.1980,F,10.03.2013,01.01.2018,41


In [9]:
personal_info_df['senior_citizen'] = personal_info_df['age'] >= 60
personal_info_df['senior_citizen'] = personal_info_df['senior_citizen'].astype(int)
personal_info_df['senior_citizen'].value_counts()

0    571
1    429
Name: senior_citizen, dtype: int64

In [10]:
personal_info_df['gender'].value_counts()

M    511
F    489
Name: gender, dtype: int64

In [11]:
personal_info_df['gender'] = LabelEncoder().fit_transform(personal_info_df['gender'])
personal_info_df

,id,birth_date,gender,subscription_start_date,subscription_end_date,age,senior_citizen
0,0,30.10.1986,0,18.08.2010,01.01.2018,34,0
1,1,22.03.1967,1,07.05.1997,01.01.2018,54,0
2,2,25.12.1965,1,23.05.2003,01.01.2018,55,0
3,3,17.06.1988,0,14.09.2013,01.01.2018,33,0
4,4,18.12.1964,1,28.01.2004,01.01.2018,56,0
...,...,...,...,...,...,...,...
995,995,08.11.1962,1,31.05.2003,01.01.2018,58,0
996,996,12.10.1987,0,18.11.2007,01.01.2018,33,0
997,997,24.10.1989,0,24.10.2007,13.12.2017,31,0
998,998,04.04.1980,0,10.03.2013,01.01.2018,41,0


In [12]:
def calculate_duration(start, end):
    start = datetime.strptime(start, "%d.%m.%Y")
    end   = datetime.strptime(end, "%d.%m.%Y") 
    return abs((end - start).days)
calculate_duration('27.07.2021', '20.07.2021')

7

In [13]:
personal_info_df['subscription_duration'] = personal_info_df.apply(lambda x: calculate_duration(x['subscription_end_date'], x['subscription_start_date']), axis=1)
personal_info_df

,id,birth_date,gender,subscription_start_date,subscription_end_date,age,senior_citizen,subscription_duration
0,0,30.10.1986,0,18.08.2010,01.01.2018,34,0,2693
1,1,22.03.1967,1,07.05.1997,01.01.2018,54,0,7544
2,2,25.12.1965,1,23.05.2003,01.01.2018,55,0,5337
3,3,17.06.1988,0,14.09.2013,01.01.2018,33,0,1570
4,4,18.12.1964,1,28.01.2004,01.01.2018,56,0,5087
...,...,...,...,...,...,...,...,...
995,995,08.11.1962,1,31.05.2003,01.01.2018,58,0,5329
996,996,12.10.1987,0,18.11.2007,01.01.2018,33,0,3697
997,997,24.10.1989,0,24.10.2007,13.12.2017,31,0,3703
998,998,04.04.1980,0,10.03.2013,01.01.2018,41,0,1758


In [14]:
personal_info_df.drop(columns=['birth_date'], inplace=True)
personal_info_df.drop(columns=['age'], inplace=True)
personal_info_df.drop(columns=['subscription_start_date'], inplace=True)
personal_info_df.drop(columns=['subscription_end_date'], inplace=True)
personal_info_df

,id,gender,senior_citizen,subscription_duration
0,0,0,0,2693
1,1,1,0,7544
2,2,1,0,5337
3,3,0,0,1570
4,4,1,0,5087
...,...,...,...,...
995,995,1,0,5329
996,996,0,0,3697
997,997,0,0,3703
998,998,0,0,1758


In [15]:
personal_info_df.describe()

,id,gender,senior_citizen,subscription_duration
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,499.500000,0.511000,0.429000,4300.654000
std,288.819436,0.500129,0.495181,3115.226943
min,0.000000,0.000000,0.000000,28.000000
25%,249.750000,0.000000,0.000000,1970.250000
50%,499.500000,1.000000,0.000000,3544.500000
75%,749.250000,1.000000,1.000000,5782.500000
max,999.000000,1.000000,1.000000,20042.000000


In [16]:
personal_info_df.to_csv(os.path.join(pwd, 'personal_info_preprocessed.csv'), index=False)

## Preprocess Contract

In [17]:
contract_info_column_names = ["id", "contract_start_date", "contract_end_date"]
contract_info_df = pd.read_csv(filepath_contract_info,  sep='\t', names=contract_info_column_names)
contract_info_df

,id,contract_start_date,contract_end_date
0,0,21.09.2015,21.03.2016
1,0,12.06.2016,12.12.2017
2,1,31.12.2016,30.06.2017
3,2,23.01.2016,23.07.2016
4,2,04.06.2017,04.06.2018
...,...,...,...
1153,997,13.01.2017,13.07.2017
1154,998,02.01.2015,02.07.2015
1155,998,18.11.2015,18.05.2016
1156,999,07.01.2016,07.01.2017


In [18]:
contract_info_df.isna().sum()

id                     0
contract_start_date    0
contract_end_date      0
dtype: int64

In [19]:
contract_info_df.groupby(['id']).count()

,contract_start_date,contract_end_date
id,,
0,2,2
1,1,1
2,2,2
3,2,2
4,1,1
...,...,...
995,1,1
996,1,1
997,1,1


In [20]:
contract_info_df_new = pd.DataFrame(range(1000), columns=['id'])
contract_info_df_new

,id
0,0
1,1
2,2
3,3
4,4
...,...
995,995
996,996
997,997
998,998


In [21]:
contract_info_df_new['number_of_contracts'] = contract_info_df.groupby(['id']).count()[['contract_start_date']]
contract_info_df_new['number_of_contracts'].isna().sum()

186

In [22]:
contract_info_df_new['number_of_contracts'].fillna(0, inplace=True)
contract_info_df_new['number_of_contracts'] = contract_info_df_new['number_of_contracts'].astype(int)
contract_info_df_new

,id,number_of_contracts
0,0,2
1,1,1
2,2,2
3,3,2
4,4,1
...,...,...
995,995,1
996,996,1
997,997,1
998,998,2


In [23]:
print('Minimum number of contracts:', contract_info_df_new['number_of_contracts'].min())
print('Maximum number of contracts:', contract_info_df_new['number_of_contracts'].max())

Minimum number of contracts: 0
Maximum number of contracts: 3


In [24]:
contract_info_df_new.to_csv(os.path.join(pwd, 'contract_info_preprocessed.csv'), index=False)

## Preprocess Invoice

In [25]:
invoice_column_names = ["id", "month", "amount"]
invoice_df = pd.read_csv(filepath_invoice,  sep='\t', names=invoice_column_names)
invoice_df

,id,month,amount
0,0,1.2015,109
1,0,1.2016,24
2,0,1.2017,60
3,0,2.2015,61
4,0,2.2016,25
...,...,...,...
35995,999,11.2016,65
35996,999,11.2017,59
35997,999,12.2015,52
35998,999,12.2016,62


In [26]:
def convert_date_to_months(date):
    date = datetime.strptime(str(date), "%m.%Y")
    return date.year * 12 + date.month
convert_date_to_months('02.0001')

14

In [27]:
invoice_df['month'] = invoice_df['month'].apply(convert_date_to_months)
invoice_df.sort_values(['id','month'], ascending=[True, True], inplace=True)
invoice_df

,id,month,amount
0,0,24181,109
3,0,24182,61
6,0,24183,67
9,0,24184,48
12,0,24185,43
...,...,...,...
35987,999,24212,33
35990,999,24213,32
35993,999,24214,71
35996,999,24215,59


In [28]:
invoice_min_df = invoice_df.groupby('id').min()[['amount']]
invoice_min_df

,amount
id,
0,17
1,22
2,19
3,20
4,18
...,...
995,19
996,19
997,7


In [29]:
invoice_min_df['churn'] = invoice_min_df[invoice_min_df['amount'] == 0]
print(invoice_min_df['churn'].isna().sum())
invoice_min_df['churn']

807


id
0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
995    NaN
996    NaN
997    NaN
998    NaN
999    0.0
Name: churn, Length: 1000, dtype: float64

In [30]:
invoice_min_df['churn'].replace(0.0, 1, inplace=True)
invoice_min_df['churn'].fillna(0, inplace=True)
invoice_min_df['churn'] = invoice_min_df['churn'].astype(int)
invoice_min_df.drop(columns=['amount'], inplace=True)
invoice_min_df

,churn
id,
0,0
1,0
2,0
3,0
4,0
...,...
995,0
996,0
997,0


In [31]:
invoice_min_df['churn'].value_counts()

0    807
1    193
Name: churn, dtype: int64

#### Aproximately 80% : 20% => Imbalanced number of samples for the two output classes.

In [32]:
invoice_df.dtypes

id        int64
month     int64
amount    int64
dtype: object

In [33]:
# Get last non-zero invoice amount
currentId = 0

for index, row in invoice_df.iterrows():
    if currentId != row['id']:
        invoice_min_df.loc[currentId, 'last_invoice'] = previous
        currentId = row['id']
    elif row['amount'] != 0 :
        previous = row['amount']
            
invoice_min_df.loc[currentId, 'last_invoice'] = previous
invoice_min_df       

,churn,last_invoice
id,,
0,0,41.0
1,0,27.0
2,0,19.0
3,0,55.0
4,0,71.0
...,...,...
995,0,19.0
996,0,53.0
997,0,7.0


In [34]:
invoice_min_df.loc[6, :] 

churn            1.0
last_invoice    15.0
Name: 6, dtype: float64

In [35]:
# Get total invoice per customer
invoice_df_total = invoice_df.groupby('id').sum()[['amount']]
invoice_df_total

,amount
id,
0,1793
1,1573
2,1793
3,1587
4,1638
...,...
995,1555
996,1576
997,1490


In [36]:
invoice_df

,id,month,amount
0,0,24181,109
3,0,24182,61
6,0,24183,67
9,0,24184,48
12,0,24185,43
...,...,...,...
35987,999,24212,33
35990,999,24213,32
35993,999,24214,71
35996,999,24215,59


In [37]:
# Get number of months with non zero invoice
counter = 0
currentId = 0

for row in invoice_df.drop(columns=['month']).iterrows():
    if row[1].id != currentId:
        invoice_df_total.loc[currentId, 'total_non_zero_months'] = counter
        currentId = row[1].id
        counter = 0
    if row[1].amount != 0:
        counter += 1
        
invoice_df_total.loc[currentId, 'total_non_zero_months'] = previous
invoice_df_total

,amount,total_non_zero_months
id,,
0,1793,36.0
1,1573,36.0
2,1793,36.0
3,1587,36.0
4,1638,36.0
...,...,...
995,1555,36.0
996,1576,36.0
997,1490,36.0


In [38]:
invoice_df_total.loc[6, :] 

amount                   150.0
total_non_zero_months      3.0
Name: 6, dtype: float64

In [39]:
(invoice_df_total['total_non_zero_months'] == 0).value_counts()

False    999
True       1
Name: total_non_zero_months, dtype: int64

In [40]:
invoice_df_new = pd.DataFrame(range(1000))
invoice_df_new.columns = ['id']
invoice_df_new['average_invoice'] = invoice_df_total['amount'] / invoice_df_total['total_non_zero_months']
invoice_df_new.fillna(0, inplace=True)
invoice_df_new['last_invoice'] = invoice_min_df['last_invoice']
invoice_df_new['churn'] = invoice_min_df['churn']
invoice_df_new

,id,average_invoice,last_invoice,churn
0,0,49.805556,41.0,0
1,1,43.694444,27.0,0
2,2,49.805556,19.0,0
3,3,44.083333,55.0,0
4,4,45.500000,71.0,0
...,...,...,...,...
995,995,43.194444,19.0,0
996,996,43.777778,53.0,0
997,997,41.388889,7.0,0
998,998,48.638889,104.0,0


In [41]:
invoice_df_new.isna().sum()

id                 0
average_invoice    0
last_invoice       0
churn              0
dtype: int64

In [42]:
invoice_df_new.describe()

,id,average_invoice,last_invoice,churn
count,1000.000000,1000.000000,1000.000000,1000.00000
mean,499.500000,44.428547,43.221000,0.19300
std,288.819436,5.183144,21.752824,0.39485
min,0.000000,0.000000,1.000000,0.00000
25%,249.750000,42.444444,28.000000,0.00000
50%,499.500000,44.902778,41.000000,0.00000
75%,749.250000,47.340278,55.000000,0.00000
max,999.000000,79.000000,160.000000,1.00000


In [43]:
invoice_df_new.to_csv(os.path.join(pwd, 'invoice_preprocessed.csv'), index=False)

## Preprocess Phone calls

In [44]:
phone_calls_column_names = ["id", "date", "duration", "operator", "roaming"]
phone_calls_df = pd.read_csv(filepath_phone_calls,  sep='\t', names=phone_calls_column_names)
phone_calls_df

,id,date,duration,operator,roaming
0,0,01.01.2015,49,1,0
1,0,01.01.2015,286,2,0
2,0,02.01.2015,899,1,0
3,0,02.01.2015,92,4,0
4,0,02.01.2015,831,12,1
...,...,...,...,...,...
1087464,999,29.12.2017,12,4,0
1087465,999,30.12.2017,525,0,0
1087466,999,30.12.2017,6,1,0
1087467,999,30.12.2017,16,3,0


In [45]:
phone_calls_df_new = pd.DataFrame(range(1000))
phone_calls_df_new.columns = ['id']
phone_calls_df_new

,id
0,0
1,1
2,2
3,3
4,4
...,...
995,995
996,996
997,997
998,998


In [46]:
total_minutes_df = phone_calls_df.groupby(['id', 'roaming']).sum()[['duration']]
total_minutes_df

duration
id  roaming          
0   0          343365
    1           38644
1   0          333573
    1           31334
2   0          351977
...               ...
997 1           31367
998 0          314591
    1           36681
999 0          233906
    1           25870

[1999 rows x 1 columns]

In [47]:
total_minutes_df = phone_calls_df.groupby(['id', 'roaming', 'operator']).sum()[['duration']]
total_minutes_df.reset_index(inplace=True)
total_minutes_df

,id,roaming,operator,duration
0,0,0,0,78714
1,0,0,1,61369
2,0,0,2,69560
3,0,0,3,70680
4,0,0,4,63042
...,...,...,...,...
24200,999,1,20,725
24201,999,1,21,3133
24202,999,1,22,558
24203,999,1,23,1292


In [48]:
total_minutes_non_roaming_df = total_minutes_df.loc[total_minutes_df['roaming'] == 0]
total_minutes_non_roaming_our_operator_df = total_minutes_non_roaming_df.loc[total_minutes_non_roaming_df['operator'] == 0]
total_minutes_non_roaming_other_operator_df = total_minutes_non_roaming_df.loc[total_minutes_non_roaming_df['operator'] != 0]

print('total_minutes_non_roaming_our_operator_df:', total_minutes_non_roaming_our_operator_df.shape[0])
print('total_minutes_non_roaming_other_operator_df:', total_minutes_non_roaming_other_operator_df.shape[0])

total_minutes_roaming_df = total_minutes_df.loc[total_minutes_df['roaming'] == 1]
total_minutes_roaming_df = total_minutes_roaming_df.loc[total_minutes_roaming_df['operator'] != 0]

print('total_minutes_roaming_df:', total_minutes_roaming_df.shape[0])

total_minutes_non_roaming_our_operator_df: 999
total_minutes_non_roaming_other_operator_df: 3998
total_minutes_roaming_df: 19208


In [49]:
total_minutes_roaming_df = total_minutes_roaming_df.groupby('id').sum()
total_minutes_roaming_df.drop(columns=['roaming', 'operator'], inplace=True)

total_minutes_non_roaming_our_operator_df   = total_minutes_non_roaming_our_operator_df.drop(columns=['roaming', 'operator'])
total_minutes_non_roaming_other_operator_df = total_minutes_non_roaming_other_operator_df.drop(columns=['roaming', 'operator'])

phone_calls_df_new['total_minutes_roaming'] = total_minutes_roaming_df
phone_calls_df_new['total_minutes_non_roaming_our_operator'] = total_minutes_non_roaming_our_operator_df.groupby('id').sum()
phone_calls_df_new['total_minutes_non_roaming_other_operator'] = total_minutes_non_roaming_other_operator_df.groupby('id').sum()

In [50]:
phone_calls_df_new.isna().sum()

id                                          0
total_minutes_roaming                       1
total_minutes_non_roaming_our_operator      1
total_minutes_non_roaming_other_operator    0
dtype: int64

In [51]:
phone_calls_df_new.fillna(0, inplace=True)
phone_calls_df_new  = phone_calls_df_new.astype(int)
phone_calls_df_new.isna().sum()

id                                          0
total_minutes_roaming                       0
total_minutes_non_roaming_our_operator      0
total_minutes_non_roaming_other_operator    0
dtype: int64

In [52]:
phone_calls_df_new['total_days'] = phone_calls_df.groupby(['id']).count()[['date']]

phone_calls_df_new['total_minutes_roaming'] = phone_calls_df_new['total_minutes_roaming'] / phone_calls_df_new['total_days']
phone_calls_df_new['total_minutes_non_roaming_our_operator'] = phone_calls_df_new['total_minutes_non_roaming_our_operator'] / phone_calls_df_new['total_days']
phone_calls_df_new['total_minutes_non_roaming_other_operator'] = phone_calls_df_new['total_minutes_non_roaming_other_operator'] / phone_calls_df_new['total_days']

phone_calls_df_new.drop(columns=['total_days'], inplace=True)
phone_calls_df_new.rename(columns={"total_minutes_roaming": "average_minutes_roaming",
                                   "total_minutes_non_roaming_our_operator": "average_minutes_non_roaming_our_operator",
                                   "total_minutes_non_roaming_other_operator": "average_minutes_non_roaming_other_operator"}, inplace=True)
phone_calls_df

,id,date,duration,operator,roaming
0,0,01.01.2015,49,1,0
1,0,01.01.2015,286,2,0
2,0,02.01.2015,899,1,0
3,0,02.01.2015,92,4,0
4,0,02.01.2015,831,12,1
...,...,...,...,...,...
1087464,999,29.12.2017,12,4,0
1087465,999,30.12.2017,525,0,0
1087466,999,30.12.2017,6,1,0
1087467,999,30.12.2017,16,3,0


In [53]:
phone_calls_df_new.to_csv(os.path.join(pwd, 'phone_calls_preprocessed.csv'), index=False)

## Preprocess SMS

Logic is the same as for file 'phone_calls.tsv'

In [54]:
sms_column_names = ["id", "date", "num", "operator", "roaming"]
sms_df = pd.read_csv(filepath_sms,  sep='\t', names=sms_column_names)
sms_df

,id,date,num,operator,roaming
0,0,01.01.2015,7,2,0
1,0,02.01.2015,4,0,0
2,0,02.01.2015,1,2,0
3,0,02.01.2015,13,23,1
4,0,03.01.2015,1,0,0
...,...,...,...,...,...
939544,999,26.12.2017,1,0,0
939545,999,26.12.2017,2,22,1
939546,999,27.12.2017,2,3,0
939547,999,31.12.2017,4,9,1


In [55]:
sms_df_new = pd.DataFrame(range(1000))
sms_df_new.columns = ['id']
sms_df_new

,id
0,0
1,1
2,2
3,3
4,4
...,...
995,995
996,996
997,997
998,998


In [56]:
total_sms_df = sms_df.groupby(['id', 'roaming', 'operator']).sum()[['num']]
total_sms_df.reset_index(inplace=True)
total_sms_df

,id,roaming,operator,num
0,0,0,0,416
1,0,0,1,452
2,0,0,2,453
3,0,0,3,407
4,0,0,4,424
...,...,...,...,...
24604,999,1,20,18
24605,999,1,21,19
24606,999,1,22,21
24607,999,1,23,41


In [57]:
total_sms_non_roaming_df = total_sms_df.loc[total_sms_df['roaming'] == 0]
total_sms_non_roaming_our_operator_df = total_sms_non_roaming_df.loc[total_sms_non_roaming_df['operator'] == 0]
total_sms_non_roaming_other_operator_df = total_sms_non_roaming_df.loc[total_sms_non_roaming_df['operator'] != 0]

print('total_sms_non_roaming_our_operator_df:', total_sms_non_roaming_our_operator_df.shape[0])
print('total_sms_non_roaming_other_operator_df:', total_sms_non_roaming_other_operator_df.shape[0])

total_sms_roaming_df = total_sms_df.loc[total_sms_df['roaming'] == 1]
total_sms_roaming_df = total_sms_roaming_df.loc[total_sms_roaming_df['operator'] != 0]

print('total_sms_roaming_df:', total_sms_roaming_df.shape[0])

total_sms_non_roaming_our_operator_df: 999
total_sms_non_roaming_other_operator_df: 3997
total_sms_roaming_df: 19613


In [58]:
total_sms_roaming_df = total_sms_roaming_df.groupby('id').sum()
total_sms_roaming_df.drop(columns=['roaming', 'operator'], inplace=True)

total_sms_non_roaming_our_operator_df   = total_sms_non_roaming_our_operator_df.drop(columns=['roaming', 'operator'])
total_sms_non_roaming_other_operator_df = total_sms_non_roaming_other_operator_df.drop(columns=['roaming', 'operator'])

sms_df_new['total_sms_roaming'] = total_sms_roaming_df
sms_df_new['total_sms_non_roaming_our_operator'] = total_sms_non_roaming_our_operator_df.groupby('id').sum()
sms_df_new['total_sms_non_roaming_other_operator'] = total_sms_non_roaming_other_operator_df.groupby('id').sum()

In [59]:
sms_df_new.isna().sum()

id                                      0
total_sms_roaming                       1
total_sms_non_roaming_our_operator      1
total_sms_non_roaming_other_operator    0
dtype: int64

In [60]:
sms_df_new.fillna(0, inplace=True)
sms_df_new = sms_df_new.astype(int)
sms_df_new

,id,total_sms_roaming,total_sms_non_roaming_our_operator,total_sms_non_roaming_other_operator
0,0,547,416,1736
1,1,488,472,1706
2,2,541,389,1628
3,3,536,424,1744
4,4,539,380,1669
...,...,...,...,...
995,995,622,489,1691
996,996,620,484,1757
997,997,564,399,1631
998,998,579,501,1804


In [61]:
sms_df_new['total_days'] = sms_df.groupby(['id']).count()[['date']]

sms_df_new['total_sms_roaming'] = sms_df_new['total_sms_roaming'] / sms_df_new['total_days']
sms_df_new['total_sms_non_roaming_our_operator'] = sms_df_new['total_sms_non_roaming_our_operator'] / sms_df_new['total_days']
sms_df_new['total_sms_non_roaming_other_operator'] = sms_df_new['total_sms_non_roaming_other_operator'] / sms_df_new['total_days']
phone_calls_df_new

sms_df_new.drop(columns=['total_days'], inplace=True)
sms_df_new.rename(columns={"total_sms_roaming": "average_sms_roaming",
                           "total_sms_non_roaming_our_operator": "average_sms_non_roaming_our_operator",
                           "total_sms_non_roaming_other_operator": "average_sms_non_roaming_other_operator"}, inplace=True)
sms_df_new

,id,average_sms_roaming,average_sms_non_roaming_our_operator,average_sms_non_roaming_other_operator
0,0,0.508364,0.386617,1.613383
1,1,0.462559,0.447393,1.617062
2,2,0.536706,0.385913,1.615079
3,3,0.498605,0.394419,1.622326
4,4,0.538462,0.379620,1.667333
...,...,...,...,...
995,995,0.569597,0.447802,1.548535
996,996,0.556553,0.434470,1.577199
997,997,0.543877,0.384764,1.572806
998,998,0.542643,0.469541,1.690722


In [62]:
sms_df_new.to_csv(os.path.join(pwd, 'sms_preprocessed.csv'), index=False)

## Preprocess data.tsv

In [63]:
data_column_names = ["id", "date", "size", "operator", "roaming"]
data_df = pd.read_csv(filepath_data,  sep='\t', names=data_column_names)
data_df

,id,date,size,operator,roaming
0,0,02.01.2015,443,21,1
1,0,03.01.2015,1,0,0
2,0,04.01.2015,21,0,0
3,0,04.01.2015,286,11,1
4,0,05.01.2015,333,19,1
...,...,...,...,...,...
740172,999,25.12.2017,35,11,1
740173,999,27.12.2017,12,0,0
740174,999,28.12.2017,19,0,0
740175,999,29.12.2017,7,7,1


In [64]:
data_df_new = pd.DataFrame(range(1000))
data_df_new.columns = ['id']
data_df_new

,id
0,0
1,1
2,2
3,3
4,4
...,...
995,995
996,996
997,997
998,998


In [65]:
total_data_df = data_df.groupby(['id', 'roaming']).sum()[['size']]
total_data_df

size
id  roaming       
0   0        37746
    1        62951
1   0        35833
    1        64405
2   0        34022
...            ...
997 1        72533
998 0        40998
    1        75783
999 0        29279
    1        57532

[2000 rows x 1 columns]

In [66]:
total_data_non_roaming = total_data_df[total_data_df.index.get_level_values('roaming').isin([0])]
total_data_non_roaming = total_data_non_roaming.droplevel("roaming")
data_df_new['total_data_non_roaming'] = total_data_non_roaming
data_df_new.isna().sum()

id                        0
total_data_non_roaming    0
dtype: int64

In [67]:
total_data_roaming = total_data_df[total_data_df.index.get_level_values('roaming').isin([1])]
total_data_roaming = total_data_roaming.droplevel("roaming")
total_data_roaming

,size
id,
0,62951
1,64405
2,67991
3,72941
4,67940
...,...
995,67783
996,85407
997,72533


In [68]:
total_data_roaming = total_data_roaming.join(phone_calls_df_new['id'], how="outer")[['size']].fillna(0).astype(int)
data_df_new['total_data_roaming'] = total_data_roaming
data_df_new

,id,total_data_non_roaming,total_data_roaming
0,0,37746,62951
1,1,35833,64405
2,2,34022,67991
3,3,32624,72941
4,4,33546,67940
...,...,...,...
995,995,35063,67783
996,996,37388,85407
997,997,35064,72533
998,998,40998,75783


In [69]:
data_df_new['total_days'] = phone_calls_df.groupby(['id']).count()[['date']]

data_df_new['total_data_non_roaming'] = data_df_new['total_data_non_roaming'] / data_df_new['total_days']
data_df_new['total_data_roaming'] = data_df_new['total_data_roaming'] / data_df_new['total_days']

data_df_new.drop(columns=['total_days'], inplace=True)
data_df_new.rename(columns={"total_data_non_roaming": "average_data_non_roaming",
                                   "total_data_roaming": "average_data_roaming"}, inplace=True)
data_df_new

,id,average_data_non_roaming,average_data_roaming
0,0,31.092257,51.854201
1,1,29.985774,53.895397
2,2,27.392915,54.743156
3,3,27.232053,60.885643
4,4,27.206813,55.101379
...,...,...,...
995,995,29.539174,57.104465
996,996,31.287029,71.470293
997,997,28.300242,58.541566
998,998,34.655959,64.060017


In [70]:
data_df_new.to_csv(os.path.join(pwd, 'data_preprocessed.csv'), index=False)

## Create final file for training

In [71]:
pwd = os.getcwd()
pwd = os.path.join(pwd, 'data')
files = glob.glob(pwd + "/*preprocessed.csv")
files

['C:\\DEV\\Projects\\Churn task\\data\\contract_info_preprocessed.csv',
 'C:\\DEV\\Projects\\Churn task\\data\\data_preprocessed.csv',
 'C:\\DEV\\Projects\\Churn task\\data\\invoice_preprocessed.csv',
 'C:\\DEV\\Projects\\Churn task\\data\\personal_info_preprocessed.csv',
 'C:\\DEV\\Projects\\Churn task\\data\\phone_calls_preprocessed.csv',
 'C:\\DEV\\Projects\\Churn task\\data\\sms_preprocessed.csv']

In [72]:
isFirst = True
for file in files:
    print(f'Processing: {file}')
    current_df = pd.read_csv(file)
    if isFirst:
        isFirst = False
        final_df = current_df
        continue
    else:
        current_df.drop(columns=['id'], inplace=True)
        final_df = pd.concat([final_df, current_df], axis=1)
final_df

Processing: C:\DEV\Projects\Churn task\data\contract_info_preprocessed.csv
Processing: C:\DEV\Projects\Churn task\data\data_preprocessed.csv
Processing: C:\DEV\Projects\Churn task\data\invoice_preprocessed.csv
Processing: C:\DEV\Projects\Churn task\data\personal_info_preprocessed.csv
Processing: C:\DEV\Projects\Churn task\data\phone_calls_preprocessed.csv
Processing: C:\DEV\Projects\Churn task\data\sms_preprocessed.csv


,id,number_of_contracts,average_data_non_roaming,average_data_roaming,average_invoice,last_invoice,churn,gender,senior_citizen,subscription_duration,average_minutes_roaming,average_minutes_non_roaming_our_operator,average_minutes_non_roaming_other_operator,average_sms_roaming,average_sms_non_roaming_our_operator,average_sms_non_roaming_other_operator
0,0,2,31.092257,51.854201,49.805556,41.0,0,0,0,2693,31.831960,64.838550,217.999176,0.508364,0.386617,1.613383
1,1,1,29.985774,53.895397,43.694444,27.0,0,1,0,7544,26.220921,62.532218,216.608368,0.462559,0.447393,1.617062
2,2,2,27.392915,54.743156,49.805556,19.0,0,1,0,5337,31.363929,57.785024,225.610306,0.536706,0.385913,1.615079
3,3,2,27.232053,60.885643,44.083333,55.0,0,0,0,1570,25.888147,61.686144,211.398164,0.498605,0.394419,1.622326
4,4,1,27.206813,55.101379,45.500000,71.0,0,1,0,5087,26.841038,48.312247,226.545012,0.538462,0.379620,1.667333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,1,29.539174,57.104465,43.194444,19.0,0,1,0,5329,23.842460,53.214827,217.833193,0.569597,0.447802,1.548535
996,996,1,31.287029,71.470293,43.777778,53.0,0,0,0,3697,23.697908,58.019247,223.001674,0.556553,0.434470,1.577199
997,997,1,28.300242,58.541566,41.388889,7.0,0,0,0,3703,25.316384,36.225989,144.425343,0.543877,0.384764,1.572806
998,998,2,34.655959,64.060017,48.638889,104.0,0,0,0,1758,31.006762,53.535080,212.391378,0.542643,0.469541,1.690722


In [73]:
final_df.isna().sum()

id                                            0
number_of_contracts                           0
average_data_non_roaming                      0
average_data_roaming                          0
average_invoice                               0
last_invoice                                  0
churn                                         0
gender                                        0
senior_citizen                                0
subscription_duration                         0
average_minutes_roaming                       0
average_minutes_non_roaming_our_operator      0
average_minutes_non_roaming_other_operator    0
average_sms_roaming                           0
average_sms_non_roaming_our_operator          0
average_sms_non_roaming_other_operator        0
dtype: int64

In [74]:
final_df.to_csv(os.path.join(pwd, 'final_file.csv'), index=False)